# Notes
- Use pip freeze to generate requirements.txt
- Group data for each year (SQL select) into a dataframe

# Requirements

* Query the dataset using sqlite. Only load the final dataset into a dataframe.

* Give an overview of the respondents of the survey. What is the sample size?
* What are the sociodemographic features of the respondents? Do you see any evidence of sampling bias?
* Perform exploratory data analysis. This should include creating statistical summaries and charts, checking for correlations and other relationships between variables, as well as other EDA elements.
* In a plot, report the prevalence rate of at least three mental diseases. (https://en.wikipedia.org/wiki/Prevalence)
* Make sure to plot the confidence interval and provide its interpretation.
* Your notebook should be readable as a standalone document. In Markdown cells inform the reader of the questions you are trying to answer, and provide an interpretation of your results.
* Provide suggestions about how your analysis can be improved.
# Questions to answer

- What are the main types and subtypes of data?
- What are the main metrics of location? What are their main characteristics?
- What is variability? What are the main metrics of variability and their characteristics?
- What is a confidence interval? Why do we need it? Why is it not sufficient to just report the point estimates?
- What is correlation? How do we use it to analyze data?
- What is a contingency table?

